# President Biden polls

### Get Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import glob
import os
import numpy as np
import requests
from bs4 import BeautifulSoup

In [3]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

### Get latest Biden trends from Real Clear Politics

In [4]:
url = "https://www.realclearpolitics.com/epolls/other/president-biden-job-approval-7320.html#polls"
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X "}
r = requests.get(url, headers=headers)

In [5]:
soup = BeautifulSoup(r.text, "html.parser")

In [6]:
tables = soup.findAll("table", attrs={"class": "data"})

In [7]:
src = pd.read_html(str(tables))[0]

In [8]:
latest_df = src[:1]

---

### Grab the latest poll trend from 538

In [9]:
trend_url = (
    "https://projects.fivethirtyeight.com/biden-approval-data/approval_topline.csv"
)

In [10]:
trend_src = pd.read_csv(trend_url)

In [11]:
trend_src.head()

,president,subgroup,modeldate,approve_estimate,approve_hi,approve_lo,disapprove_estimate,disapprove_hi,disapprove_lo,timestamp
0,Joe Biden,All polls,12/22/2021,43.393962,47.918782,38.869142,51.437616,56.700127,46.175106,12:21:14 22 Dec 2021
1,Joe Biden,Adults,12/22/2021,43.196296,48.023481,38.369112,50.168691,54.772466,45.564916,12:21:18 22 Dec 2021
2,Joe Biden,Voters,12/22/2021,43.475270,47.675131,39.275409,51.841684,56.928275,46.755093,12:21:21 22 Dec 2021
3,Joe Biden,All polls,12/21/2021,43.664251,48.167267,39.161235,51.751454,56.896447,46.606460,20:53:13 21 Dec 2021
4,Joe Biden,Adults,12/21/2021,43.714489,48.332894,39.096084,50.444127,55.175257,45.712997,20:53:17 21 Dec 2021


In [12]:
trend_src.to_csv("data/processed/five_thirty_eight_approval_topline.csv", index=False)

### All the polls from 538

In [13]:
polls_url = (
    "https://projects.fivethirtyeight.com/biden-approval-data/approval_polllist.csv"
)

In [14]:
polls_src = pd.read_csv(polls_url)

In [15]:
polls_src.head()

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,influence,approve,disapprove,adjusted_approve,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp
0,Joe Biden,All polls,12/22/2021,1/19/2021,1/21/2021,Rasmussen Reports/Pulse Opinion Research,B,1500.0,lv,0.338188,0.0,48.0,45.0,50.248618,39.001539,NaN,T,https://www.rasmussenreports.com/public_conten...,74247,139395,1/22/2021,12:21:14 22 Dec 2021
1,Joe Biden,All polls,12/22/2021,1/19/2021,1/21/2021,Morning Consult,B,15000.0,a,0.259361,0.0,50.0,28.0,48.513028,31.235697,NaN,T,https://morningconsult.com/form/global-leader-...,74272,139491,1/28/2021,12:21:14 22 Dec 2021
2,Joe Biden,All polls,12/22/2021,1/20/2021,1/21/2021,Ipsos,B-,1115.0,a,1.101398,0.0,55.0,32.0,53.640076,33.315150,NaN,NaN,https://www.ipsos.com/sites/default/files/ct/n...,74248,139404,1/22/2021,12:21:14 22 Dec 2021
3,Joe Biden,All polls,12/22/2021,1/20/2021,1/21/2021,YouGov,B+,1516.0,a,1.245401,0.0,45.0,28.0,46.387706,28.588919,NaN,NaN,https://docs.cdn.yougov.com/u3h9dresbn/2021012...,74327,139570,2/2/2021,12:21:14 22 Dec 2021
4,Joe Biden,All polls,12/22/2021,1/20/2021,1/21/2021,Morning Consult,B,1993.0,rv,0.092978,0.0,56.0,31.0,54.513028,34.235697,NaN,NaN,https://assets.morningconsult.com/wp-uploads/2...,74246,139394,1/22/2021,12:21:14 22 Dec 2021


In [16]:
polls_src.to_csv("data/processed/five_thirty_eight_approval_polllist.csv", index=False)